## 1. Load required libraries

In [2]:
import os
import json
import gzip
import umap
import hdbscan

## 2. Define UMAP / HDBSCAN functions

https://github.com/dborrelli/chat-intents

1.  n_neighbors

* A suitable starting range for n_neighbors with sentence embeddings is between 5 and 50. Here's why:

  * Lo*wer bound (5): Prevents overly localized clusters and ensures a minimum level of connectivity between data points.
  * Upper bound (50): Avoids creating clusters that are too broad and lack meaningful semantic distinctions.

* Factors Affecting the Range

  * Dataset size: Larger datasets might tolerate slightly larger n_neighbors ranges before clusters become too broad.
  * Embedding quality: Well-trained embeddings with clear semantic distinctions might benefit from slightly smaller n_neighbors 2. 
2.  min_dist

* A suitable starting range for min_dist is between 0.0 and 0.5.  Here's the reasoning
  * Lower bound (0.0): Allows for the possibility of dense, overlapping clusters if the data structure warrants it.
  * Upper bound (0.5): Prevents clusters from becoming excessively compressed, aiding visual interpretation and potentially improving downstream clustering results.

* Generally, you would use a lower min_dist with a smaller n_neighbors and vice versa.

3.  n_components

* A suitable starting range for n_components is between 5 and 15. Here's the reasoning:
* Minimum for HDBSCAN: HDBSCAN benefits from having some room to identify varying densities within the data. A minimum of around 5 dimensions generally provides enough space to form more nuanced clusters.

4.  metric

* A suitable metric is cosine.

  * Cosine: Cosine distance is particularly suitable for sentence embedding clustering due to its focus on directionality, inherent normalization, and sparsity preservation.

  * Mahalanobis: This metric takes into account the covariance structure of the data, making it effective when the data exhibits different variances or correlations across features. While it might not be the most intuitive choice for sentence embeddings, it can be a good option if the embeddings have complex relationships between features.

  * Euclidean: This is the default metric used by UMAP and is a common choice for many clustering tasks. However, for sentence embeddings, it might not be as effective as cosine distance because it prioritizes absolute differences in magnitudes which might not capture semantic similarity well.

  * Minkowski (including Manhattan and Chebyshev): These metrics are generalizations of Euclidean distance, where Manhattan distance uses the sum of absolute differences and Chebyshev distance uses the maximum absolute difference. While they might be suitable for specific scenarios, they generally lack the interpretability and effectiveness of cosine distance for sentence embeddings.

  * Miscellaneous spatial metrics (Canberra, Braycurtis, Haversine): These metrics are designed for specific purposes like measuring similarity between categorical data (Canberra, Braycurtis) or geographical locations (Haversine). They are not well-suited for the task of sentence embedding clustering.

  * Normalized spatial metrics (wminkowski, seuclidean): These are variants of Minkowski distance with some normalization applied. While they might address some limitations of the original metrics, they generally don't offer significant advantages over cosine distance for sentence embeddings.

  * Angular and correlation metrics (Correlation): While correlation measures the linear relationship between vectors, it might not be suitable for capturing semantic similarity in sentence embeddings, which can have complex non-linear relationships.

  * Metrics for binary data: These metrics (Hamming, Jaccard, Dice, etc.) are designed to compare binary data and are not applicable to sentence embeddings, which are typically represented as continuous vector representations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA


def pca_preprocessor(data, plot=False):
  n_components_max = len(data)
  pca = PCA(n_components=n_components_max)
  pca.fit(data)

  explained_variance_ratio_ = pca.explained_variance_ratio_
  cumulative_explained_variance_ratio_ = explained_variance_ratio_.cumsum()

  if plot:
    # Plot explained variance
    plt.figure()
    plt.plot(range(1, n_components_max + 1), explained_variance_ratio_)
    plt.plot(range(1, n_components_max + 1), cumulative_explained_variance_ratio_)
    plt.xlabel("Number of Components")
    plt.ylabel("Explained Variance Ratio")
    plt.title("Explained Variance by PCA")
    plt.show()

  # Calculate the second derivative
  second_derivative = np.gradient(np.gradient(cumulative_explained_variance_ratio_))
  # Find the index of the maximum of the second derivative
  recommended_components_elbow = np.argmax(second_derivative)

  # Alternative recommendation based on 90% explained variance
  threshold = 0.9
  recommended_components_90_pct = None  # Initialize outside the loop
  for i, ratio in enumerate(cumulative_explained_variance_ratio_):
    if ratio >= threshold:
      recommended_components_90_pct = i + 1
      break

  print(f"Recommended number of components (elbow method): {recommended_components_elbow}")
  print(f"Recommended number of components (90% threshold): {recommended_components_90_pct}")
  
  selected_components = min(recommended_components_elbow, recommended_components_90_pct)
  pca = PCA(n_components=selected_components)
  print(f"Selected number of components: {selected_components}")

  return pca.fit_transform(data)


def umap_wrapper(n_neighbors, min_dist, n_components, metric):
    return umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric
        )


def umap_reduce_2d(data, n_neighbors=15, min_dist=0.1, n_components=2, metric='cosine', plot2d=True, plot_diagnostics=True, preprocess_pca=True):
    reducer = umap_wrapper(n_neighbors=n_neighbors,
                           min_dist=min_dist,
                           n_components=n_components,
                           metric=metric
                           )
    if preprocess_pca:
       data = pca_preprocessor(data)
    reduced_data = reducer.fit_transform(data)
    
    if plot2d:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.scatter(reduced_data[:,0], reduced_data[:,1])
        plt.title(label=f"n_neighbors={n_neighbors} - min_dist={min_dist} - n_components={n_components} - metric={metric}", fontsize=18)
    
    if plot_diagnostics:
        # The "hammer" algorithm, in this context, bundles together similar connections (lines) in the visualization. 
        # This declutters the plot, making it easier to see the overall trends and major connections within the UMAP embedding.  
        umap.plot.connectivity(reducer, edge_bundling='hammer')
        umap.plot.diagnostic(reducer, diagnostic_type='pca')
        umap.plot.diagnostic(reducer, diagnostic_type='vq')
        # When the local dimension is high it represents points that UMAP will have a harder time embedding as well.
        # While the local dimension plot can provide some insights, it cannot be solely relied upon to estimate the optimal number of dimensions for UMAP
        local_dims = umap.plot.diagnostic(reducer, diagnostic_type='local_dim')
    return reduced_data


def umap_reduce(data, n_neighbors=30, min_dist=0.0, n_components=10, metric='cosine', preprocess_pca=True):
    """
    When using UMAP for dimension reduction we want to select different parameters than if you were using it for visualization. 
    First of all we want a larger n_neighbors value, small values are more prone to producing fine grained cluster structure 
    that may be more a result of patterns of noise in the data than actual clusters. 
    Second it is beneficial to set min_dist to a very low value. Since we actually want to pack points together densely 
    a low value will help, as well as making cleaner separations between clusters.
    """
    reducer = umap_wrapper(n_neighbors=n_neighbors,
                           min_dist=min_dist,
                           n_components=n_components,
                           metric=metric
                           )
    
    if preprocess_pca:
       data = pca_preprocessor(data)
    return reducer.fit_transform(data)


In [5]:


# read file
def get_most_recent_file(directory: str) -> str | None:
    files = [os.path.join(directory, f) for f in os.listdir(directory)]
    files = [f for f in files if os.path.isfile(f)]
    if not files:
        return None
    file_ctimes = [(f, os.path.getctime(f)) for f in files]
    most_recent_file = sorted(file_ctimes, key=lambda x: x[1], reverse=True)[0][0]
    return most_recent_file


# fit_transform umap at optimal settings
def umap_reduce(embeddings):
    reducer = umap.UMAP(n_neighbors=30, min_dist=1, n_components=20, metric="cosine")
    return reducer.fit_transform(embeddings)


# fit_transform hdbscan at optimal settings
def hdbscan_clustering(reduced_embeddings):
    clusterer = hdbscan.HDBSCAN(min_cluster_size=10,
                                min_samples=5,
                                cluster_selection_epsilon=0.0,
                                metric="euclidean",
                                cluster_selection_method="eom", # leaf
                                prediction_data=True)
    return clusterer.fit_predict(reduced_embeddings)


# fit_transform umap to 2d
def umap_reduce_2d(embeddings):
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="cosine")
    return reducer.fit_transform(embeddings)

# apply cluster label to entries
# save plots for git pages

def daily_processing(filename=None):
    if filename is None:
        filename = get_most_recent_file("../data")
    with gzip.open(filename, "r") as file:
        arxiv = json.loads(file.read().decode("utf-8"))
        embeddings = [d["embedding"] for d in arxiv]
        reduced_embeddings = umap_reduce(embeddings)
        labels = hdbscan_clustering(reduced_embeddings)
        reduced_embeddings_2d = umap_reduce_2d(embeddings)
    for reduced_embedding_2d, entry in zip(reduced_embeddings_2d, arxiv):
        entry["plot_embedding"] = reduced_embedding_2d
    for label, entry in zip(labels, arxiv):
        entry["cluster"] = label
        

daily_processing()

labels: [ 5  3  5  5 -1 -1 -1  1  5 -1  5  4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  2 -1 -1 -1  1 -1 -1 -1  4 -1  2 -1 -1  4 -1 -1 -1  3  3  4 -1 -1
 -1 -1 -1 -1 -1 -1 -1  5  4 -1 -1 -1 -1 -1 -1  3 -1 -1 -1  5 -1  2  2  3
 -1 -1  5 -1 -1 -1 -1 -1  4  5  1 -1 -1  5 -1 -1  2  4 -1 -1 -1 -1 -1  2
  5  5 -1 -1 -1  2  1  5 -1 -1  2 -1  2  5 -1 -1  5  3  4  4 -1 -1  4 -1
  0 -1 -1 -1  2 -1 -1  2 -1 -1 -1  2  2 -1 -1 -1 -1  4 -1 -1  2 -1 -1 -1
  2  5  0 -1 -1  5 -1 -1 -1  2 -1 -1 -1 -1 -1  5  2 -1 -1 -1 -1 -1  2  2
 -1  3 -1  2 -1 -1 -1 -1 -1  0 -1 -1  1 -1  5  5  5 -1 -1  2  2  2 -1  0
  5  5 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  5 -1 -1 -1  4  3
 -1 -1  1  0  2 -1  0  3  5  3  0  2 -1 -1  3 -1  3  2  4 -1  3  3 -1  4
 -1  5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1  5 -1 -1 -1  1  2 -1  0 -1
 -1 -1  3 -1 -1 -1 -1 -1  3 -1 -1  1 -1 -1  4  4  5  5  3 -1 -1 -1 -1 -1
 -1 -1  3 -1 -1 -1 -1 -1 -1 -1  3  0 -1 -1 -1  3 -1 -1  3  3  1  2 -1  5
 -1  3 -1 -1 -1 -1  2  3  5 -1  2 -1  5  5 